# Dewan Negara Data

In [1]:
#links
parlimen_15 = 'https://www.parlimen.gov.my/ahli-dewan-dn.html?uweb=dn&vol=15'
parlimen_14 = 'https://www.parlimen.gov.my/arkib-ahli.html?uweb=dn&arkib=yes&vol=14'
parlimen_13 = 'https://www.parlimen.gov.my/arkib-ahli.html?uweb=dn&arkib=yes&vol=13'
parlimen_12 = 'https://www.parlimen.gov.my/arkib-ahli.html?uweb=dn&arkib=yes&vol=12'

parlimens = [parlimen_15,parlimen_14, parlimen_13, parlimen_12]

In [2]:
from requests_html import HTMLSession
import pandas as pd
from pprint import pprint
import re
from urllib.parse import quote
from tqdm import tqdm


In [3]:
# Get all the profile links
with HTMLSession() as sess:
    r = sess.get(parlimen_14)
    profile_links = r.html.xpath('//*[@id="contentBoxText"]/ul[@class="list tiles member-of-parliament"]//a')

## 14 - 12

In [4]:

dn_dict = {
    "vol":[],
    "tempoh": [], 
    "lantikan": [], 
    "kod_kawasan": [], 
    "negeri": [], 
    "kawasan": [],
    "nama": [],
    "tahun_lahir": [], 
    "jantina": [], 
    "kumpulan_etnik": [], 
    "parti": [], 
    "jawatan_kabinet": [], 
    "jawatan_parlimen": [],
    "gambar": []
}

for parlimen in tqdm(parlimens):
    
    # Get all the profile links
    with HTMLSession() as sess:
        r = sess.get(parlimen)
        profile_links = r.html.xpath('.//ul[@class="list tiles member-of-parliament"]//a')
    
    # Get the profile data
    with HTMLSession() as sess:
        for profile in tqdm(profile_links,desc="Parlimen Profiles"):
            fields_found = set()
            
            r2 = sess.get("https://www.parlimen.gov.my/" + profile.attrs["href"])
            r2 = r2.html.xpath('.//div[@class="parfont"]')
            
            # Handle image First as they're alwasy at the top
            image = r2[0].xpath('.//tr[1]//img/@src')
            dn_dict["gambar"].append("https://www.parlimen.gov.my"+quote(image[0]) if image else None)
            dn_dict["vol"].append(parlimen.split('=')[-1])
            fields_found.add("gambar")
            fields_found.add("vol")
            
            # Process each row
            
            """
            Caveats:
            - Not all profiles have the fields we need
            - Some fields are not in the same order
            """
            for tr in r2[0].xpath('.//tr')[4:]:
                tr_key = tr.xpath('.//td[1]//text()')[0] # Get the key in the first column
                tr_value = tr.xpath('.//td[2]/text()') # Get all text in the second column
                
                if tr_key.lower() == 'lantikan semula': # Skip this row
                    continue
                
                if tr_key.lower() == 'jawatan dalam parlimen': # Skip this row
                    dn_dict["jawatan_parlimen"].append(tr_value[0] if tr_value else None)
                    fields_found.add('jawatan_parlimen')
                    
                
                if tr_key.lower() == 'jawatan dalam kabinet': # Skip this row
                    dn_dict["jawatan_kabinet"].append(tr_value[0] if tr_value else None)
                    fields_found.add('jawatan_kabinet')
                    
                    
                # Check each dictionary key against the row key
                for key in dn_dict:
                    if key in ("gambar","jawatan_parlimen","vol","jawatan_kabinet"): # Skip these keys
                        continue
                        
                    if key in tr_key.lower():
                        # print(key, tr_value)
                        dn_dict[key].append(tr_value[0].strip() if tr_value else None)
                        fields_found.add(key)
            
            # Add None for any fields not found in this profile
            for key in dn_dict:
                if key not in fields_found:
                    dn_dict[key].append(None)

# pprint(dn_dict)
dn_df = pd.DataFrame(dn_dict)
dn_df.head()

  0%|          | 0/4 [00:00<?, ?it/s]


KeyboardInterrupt: 

# Combined

In [18]:
dn_df

NameError: name 'dn_df' is not defined

In [17]:
dn_df.to_csv("./data/ahli_dewan.csv", index=False)

NameError: name 'dn_df' is not defined

# Open CSV

In [200]:
import pandas as pd

In [201]:
import re

df_dn_csv = pd.read_csv("./data/ahli_dewan.csv")

df_dn_csv['tempoh_mula'] = df_dn_csv['tempoh'].astype(str).map(lambda x: re.split(r' - | hingga|-', x)[0] if len(re.split(r' - | hingga|-', x)) > 1 else None)
df_dn_csv['tempoh_tamat'] = df_dn_csv['tempoh'].astype(str).map(lambda x: re.split(r' - | hingga|-', x)[-1] if len(re.split(r' - | hingga|-', x)) > 1 else None)

df_dn_csv['gelaran'] = df_dn_csv['nama'].map(lambda x: re.split(r',', x)[1] if len(re.split(r',', x)) > 1 else None)
df_dn_csv['nama'] = df_dn_csv['nama'].map(lambda x: re.split(r',', x)[0])

df_dn_csv['jantina'] = df_dn_csv['nama'].map(
    lambda x: "M" if any(substring in x.lower() for substring in ["bin", "a/l"]) 
                    else "P" if any(substring in x.lower() for substring in ["binti", "a/p"]) 
                    else df_dn_csv[df_dn_csv['nama'] == x]['jantina'].values[0]
).replace("P","F")

df_dn_csv['kumpulan_etnik'] = df_dn_csv['nama'].map(
    lambda x: "bumiputera" if any(substring in x.lower() for substring in ["bin", "binti"]) 
                    else "indian" if any(substring in x.lower() for substring in ["a/l", "a/p"]) 
                    else df_dn_csv[df_dn_csv['nama'] == x]['kumpulan_etnik'].values[0] 

).fillna("chinese")

df_dn_csv['lantikan'] = df_dn_csv['lantikan'].replace("0", None)
df_dn_csv['jawatan_kabinet'] = df_dn_csv['jawatan_kabinet'].fillna("TIADA").replace("Tiada", "TIADA")
df_dn_csv['jawatan_parlimen'] = df_dn_csv['jawatan_parlimen'].fillna("TIADA").replace("Tiada", "TIADA")

df_dn_csv = df_dn_csv.reindex(
    columns=[
        'vol','nama','gelaran','parti','lantikan',
        'negeri','kawasan','kod_kawasan',
        'tahun_lahir','jantina','kumpulan_etnik',
        'jawatan_kabinet','jawatan_parlimen',
        'penggal','tempoh','tempoh_mula','tempoh_tamat'
    ]
)


df_dn_csv

,vol,nama,gelaran,parti,lantikan,negeri,kawasan,kod_kawasan,tahun_lahir,jantina,kumpulan_etnik,jawatan_kabinet,jawatan_parlimen,penggal,tempoh,tempoh_mula,tempoh_tamat
0,15,Awang Bemee bin Awang Ali Basah,YB Senator Dato,PBB,Dilantik Yang di-Pertuan Agong,NaN,NaN,NaN,NaN,M,bumiputera,TIADA,Yang di-Pertua Dewan Negara,NaN,15.07.2024 - 14.07.2027,15.07.2024,14.07.2027
1,15,Nur Jazlan bin Tan Sri Mohamed,YB Senator Datuk,UMNO,Dilantik Yang di-Pertuan Agong,NaN,NaN,NaN,NaN,M,bumiputera,TIADA,Timbalan Yang di-Pertua Dewan Negara,NaN,15.06.2023 - 14.06.2026,15.06.2023,14.06.2026
2,15,Saifuddin Nasution bin Ismail,YB Senator Datuk Seri,PKR,Dilantik Yang di-Pertuan Agong,NaN,NaN,NaN,NaN,M,bumiputera,Menteri Dalam Negeri,Menteri Dalam Negeri,NaN,03.12.2022 - 02.12.2025,03.12.2022,02.12.2025
3,15,Tengku Zafrul bin Tengku Abdul Aziz,YB Senator Datuk Seri Utama,UMNO,Dilantik Yang di-Pertuan Agong,NaN,NaN,NaN,NaN,M,bumiputera,"Menteri Pelaburan, Perdagangan dan Industri","Menteri Pelaburan, Perdagangan dan Industri",NaN,09.03.2020 - 03.11.2022,09.03.2020,03.11.2022
4,15,Zambry bin Abdul Kadir,YB Senator Dato' Seri Diraja Dr.,UMNO,Dilantik Yang di-Pertuan Agong,NaN,NaN,NaN,NaN,M,bumiputera,Menteri Pendidikan Tinggi,Ahli Dewan Negara,NaN,03.12.2022 - 02.12.2025,03.12.2022,02.12.2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,12,Jamilah@Halimah binti Sulaiman,YB Senator Datuk,BN - PBS,Dilantik Yang di-Pertuan Agong,NaN,NaN,NaN,NaN,M,bumiputera,TIADA,TIADA,NaN,07.11.2012 - 06.11.2015,07.11.2012,06.11.2015
198,12,Datuk Zamani Bin Hj. Sulaiman,None,NaN,None,NaN,NaN,NaN,NaN,M,bumiputera,TIADA,TIADA,NaN,NaN,None,None
199,12,Abu Zahar Ujang,YB Senator Tan Sri,NaN,None,NaN,NaN,NaN,NaN,M,bumiputera,TIADA,TIADA,NaN,NaN,None,None
200,12,Lucas bin Umbul,YB Senator Dr.,BN - UPKO,Dipilih oleh Dewan Undangan Negeri,Sabah,NaN,NaN,NaN,M,bumiputera,TIADA,TIADA,NaN,10.12.2012 - 09.12.2015,10.12.2012,09.12.2015


In [202]:
df_dn_csv.to_csv("./data/ahli_dewan_mod.csv", index=False)

In [203]:
df_dn_csv['gelaran'].drop_duplicates().dropna().to_csv("./data/gelaran.csv", index=False)

# DN 1 -9

In [204]:
links = [f"https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-{num}" for num in range(2,12)]
links = ["https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_pertama"]+ links
pprint(links)

['https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_pertama',
 'https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-2',
 'https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-3',
 'https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-4',
 'https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-5',
 'https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-6',
 'https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-7',
 'https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-8',
 'https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-9',
 'https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-10',
 'https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-11']


## Pertama

In [205]:
import pandas as pd
from io import StringIO
from datetime import datetime, timezone, timedelta
import html

def format_parti(text:str) -> str:
    patterns = [
        r'(\w+)\s*\((\w+)\)',
        r'(\w+)\s*\|\s*\((\w+)\)',
    ]
    
    for pattern in patterns:
        match = re.match(pattern, text)
        if match:
            return f"{match.group(1)} - {match.group(2)}"
    
    return text

def date_parser(date_string):
    month_map = {
        "januari": "01", "january": "01",
        "februari": "02", "february": "02",
        "mac": "03", "march": "03",
        "april": "04",
        "mei": "05", "may": "05",
        "jun": "06", "june": "06",
        "julai": "07", "july": "07",
        "ogos": "08", "august": "08",
        "september": "09",
        "oktober": "10", "october": "10",
        "november": "11",
        "disember": "12", "december": "12"
    }

    # Unescape any HTML entities and normalize whitespace
    if isinstance(date_string,float):
        return None
    
    date_string = html.unescape(date_string)
    date_string = re.sub(r'\s+', ' ', date_string).strip()  # Replace multiple spaces with a single space
    date_string = re.sub(r'\[\d+\]', ' ', date_string).strip()  # Replace multiple spaces with a single space
    
    
    parts = date_string.split(" ")
    
    if len(parts) == 2:  # Format: "Month Year"
        month = month_map.get(parts[0].lower())  # Convert to lowercase for lookup
        year = parts[1]  # Get year
        day = "01"  # Default to the first day
    elif len(parts) == 3:  # Format: "Day Month Year"
        day = parts[0].zfill(2)  # Pad day with zero if needed
        month = month_map.get(parts[1].lower())  # Convert to lowercase for lookup
        year = parts[2]  # Get year
    elif len(parts) == 1:  # Format: "Year"
        day = "01"  # Default to the first day
        month = "01"
        year = parts[0]  # Get year
    else:
        raise ValueError(f"Date format '{date_string}' is not recognized.")

    if month is None:
        raise ValueError(f"Month '{parts[0].lower() if len(parts) == 2 else parts[1].lower()}' is not recognized.")
    
    date = datetime.strptime(f"{year}-{month}-{day}", "%Y-%m-%d")

    return date

lantikan_ydpa = {
    "vol":[],
    "nama": [],
    "parti": [],
    "penggal": [],
    "penggal_mula": [],
    "penggal_tamat": [],
    "catatan": []
}

with HTMLSession() as sess:
    r = sess.get(links[0])
    lantikans = [x for x in r.html.xpath('.//table//caption//text()') if x.strip() != ""]
    
    table_2 = r.html.xpath('.//table')[3]
    data = []
    
    lantikan_ydpa = pd.read_html(StringIO(table_2.html))[0]
    lantikan_ydpa['lantikan'] = lantikans[0]
    
with HTMLSession() as sess:
    r = sess.get(links[0])
    lantikans = [x for x in r.html.xpath('.//table//caption//text()') if x.strip() != ""]
    
    table_2 = r.html.xpath('.//table')[4]
    data = []
    
    lantikan_dun = pd.read_html(StringIO(table_2.html))[0]
    lantikan_dun['lantikan'] = lantikans[1]


lantikan_ydpa = lantikan_ydpa.drop(axis=0, columns=["No."])
lantikan_ydpa = lantikan_ydpa.iloc[lantikan_ydpa.index != 19].reset_index(drop=True)[:21]
lantikan_ydpa.columns = ['nama','parti','penggal','tempoh_mula','tempoh_tamat','catatan','lantikan']
lantikan_ydpa['lantikan'] = 'Dilantik Yang di-Pertuan Agong'
lantikan_ydpa['negeri'] = None
lantikan_ydpa['jawatan_parlimen'] = 'Ahli Dewan Negara'
lantikan_ydpa['jawatan_kabinet'] =None
lantikan_ydpa.reindex(columns=['negeri','name','parti','penggal','tempoh_mula','tempoh_tamat','lantikan','catatan'])


lantikan_dun = lantikan_dun.drop(axis=0, columns=["No."])
lantikan_dun = lantikan_dun.iloc[lantikan_dun.index != 27]
lantikan_dun.columns = ['negeri','nama','parti','penggal','tempoh_mula','tempoh_tamat','lantikan']
lantikan_dun['catatan'] = None
lantikan_dun['lantikan'] = 'Dipilih oleh Dewan Undangan Negeri'
lantikan_dun['jawatan_parlimen'] = 'Ahli Dewan Negara'
lantikan_dun['jawatan_kabinet'] =None
lantikan_dun.reindex(columns=['negeri','name','parti','penggal','tempoh_mula','tempoh_tamat','lantikan','catatan'])

combined_data = pd.concat([lantikan_ydpa, lantikan_dun], axis=0)
combined_data = combined_data.reset_index(drop=True).drop(axis=0, columns=["catatan","penggal"])
combined_data['vol'] = 1
combined_data[['parti', 'negeri']] = combined_data[['parti', 'negeri']].fillna('TIADA')


combined_data['parti'] = combined_data['parti'].apply(
    lambda x: format_parti(x)
)

combined_data['nama'] = combined_data['nama'].apply(
    lambda x: re.sub(r'\[\d+\]', '', x).strip()
)
combined_data['tempoh_mula'] = combined_data['tempoh_mula'].apply(date_parser)
combined_data['tempoh_tamat'] = combined_data['tempoh_tamat'].apply(date_parser)

combined_data = combined_data.reindex(
    columns=[
        'vol','nama','gelaran','parti','lantikan',
        'negeri','kawasan','kod_kawasan',
        'tahun_lahir','jantina','kumpulan_etnik',
        'jawatan_kabinet','jawatan_parlimen',
        'penggal','tempoh','tempoh_mula','tempoh_tamat'
    ]
)


combined_data


,vol,nama,gelaran,parti,lantikan,negeri,kawasan,kod_kawasan,tahun_lahir,jantina,kumpulan_etnik,jawatan_kabinet,jawatan_parlimen,penggal,tempoh,tempoh_mula,tempoh_tamat
0,1,A. M. Abu Bakar,NaN,Perikatan - UMNO,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1959-09-11,1962-09-10
1,1,Aishah Ghani,NaN,Perikatan - UMNO,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1962-10-15,1968-10-14
2,1,Abdul Hamid Mahmud,NaN,Perikatan - UMNO,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1959-09-11,1962-09-10
3,1,Mohamed Ghazali Jawi,NaN,Perikatan - UMNO,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1963-06-03,1964-01-01
4,1,Cheah Toon Lok,NaN,Perikatan - MCA,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1959-09-11,1965-09-10
5,1,Choo Kok Leong,NaN,Perikatan - MCA,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1959-09-11,1965-09-10
6,1,E. E. C. Thuraisingham,NaN,TIADA,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1959-09-11,1965-09-10
7,1,Engku Muhsein Abdul Kadir,NaN,Perikatan - UMNO,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1959-09-11,1964-01-01
8,1,G. Shelley,NaN,TIADA,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1959-09-11,1965-09-10
9,1,James Edward Seymour Crawford,NaN,TIADA,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1959-09-11,1965-09-10


In [206]:
combined_data.to_csv("./data/dn_wiki/dn_1.csv", index=False)

## Ke 2

In [207]:
import pandas as pd
from io import StringIO
from datetime import datetime, timezone, timedelta
import html

lantikan_ydpa = {
    "vol":[],
    "nama": [],
    "parti": [],
    "penggal": [],
    "penggal_mula": [],
    "penggal_tamat": [],
    "catatan": []
}

link = links[1]


with HTMLSession() as sess:
    r = sess.get(link)
    lantikans = [x for x in r.html.xpath('.//table//caption//text()') if x.strip() != ""]
    
    table_2 = r.html.xpath('.//table')[3]
    data = []
    
    lantikan_ydpa = pd.read_html(StringIO(table_2.html))[0]
    lantikan_ydpa['lantikan'] = lantikans[0]
    
with HTMLSession() as sess:
    r = sess.get(link)
    lantikans = [x for x in r.html.xpath('.//table//caption//text()') if x.strip() != ""]
    
    table_2 = r.html.xpath('.//table')[4]
    data = []
    
    lantikan_dun = pd.read_html(StringIO(table_2.html))[0]
    lantikan_dun['lantikan'] = lantikans[1]

# lantikan_ydpa


lantikan_ydpa = lantikan_ydpa.drop(axis=0, columns=["No."])
lantikan_ydpa = lantikan_ydpa.loc[lantikan_ydpa.index != 41]
lantikan_ydpa.columns = ['nama','parti','penggal','tempoh_mula','tempoh_tamat','lantikan']
lantikan_ydpa['lantikan'] = 'Dilantik Yang di-Pertuan Agong'
lantikan_ydpa['catatan'] = None
lantikan_ydpa['negeri'] = None
lantikan_ydpa['jawatan_parlimen'] = 'Ahli Dewan Negara'
lantikan_ydpa['jawatan_kabinet'] = None
lantikan_ydpa.reindex(columns=['negeri','name','parti','penggal','tempoh_mula','tempoh_tamat','lantikan','catatan'])

# lantikan_ydpa

lantikan_dun = lantikan_dun.drop(axis=0, columns=["No."])
lantikan_dun = lantikan_dun.iloc[lantikan_dun.index != 27]
lantikan_dun.columns = ['negeri','nama','parti','penggal','tempoh_mula','tempoh_tamat','lantikan']
lantikan_dun['catatan'] = None
lantikan_dun['lantikan'] = 'Dipilih oleh Dewan Undangan Negeri'
lantikan_dun['jawatan_parlimen'] = 'Ahli Dewan Negara'
lantikan_dun['jawatan_kabinet'] = None
lantikan_dun.reindex(columns=['negeri','name','parti','penggal','tempoh_mula','tempoh_tamat','lantikan','catatan'])

combined_data = pd.concat([lantikan_ydpa, lantikan_dun], axis=0)
combined_data = combined_data.reset_index(drop=True).drop(axis=0, columns=["catatan","penggal"])
combined_data['vol'] = 2
combined_data[['parti', 'negeri']] = combined_data[['parti', 'negeri']].fillna('TIADA')


combined_data['parti'] = combined_data['parti'].apply(
    lambda x: format_parti(x)
)

combined_data['nama'] = combined_data['nama'].apply(
    lambda x: re.sub(r'\[\d+\]', '', x).strip()
)

combined_data['tempoh_mula'] = combined_data['tempoh_mula'].apply(date_parser)
combined_data['tempoh_tamat'] = combined_data['tempoh_tamat'].apply(date_parser)

combined_data = combined_data.reindex(
    columns=[
        'vol','nama','gelaran','parti','lantikan',
        'negeri','kawasan','kod_kawasan',
        'tahun_lahir','jantina','kumpulan_etnik',
        'jawatan_kabinet','jawatan_parlimen',
        'penggal','tempoh','tempoh_mula','tempoh_tamat'
    ]
)


combined_data.to_csv("./data/dn_wiki/dn_2.csv", index=False)
combined_data



,vol,nama,gelaran,parti,lantikan,negeri,kawasan,kod_kawasan,tahun_lahir,jantina,kumpulan_etnik,jawatan_kabinet,jawatan_parlimen,penggal,tempoh,tempoh_mula,tempoh_tamat
0,2,Cheah Toon Lok,NaN,Perikatan - MCA,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1959-09-11,NaT
1,2,Cheah Toon Lok,NaN,Perikatan - MCA,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1968-10-22,NaT
2,2,Choo Kok Leong,NaN,Perikatan - MCA,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1959-09-11,1965-08-16
3,2,Chan Keong Hon,NaN,Perikatan - MCA,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1965-12-18,NaT
4,2,E. E. C. Thuraisingham,NaN,TIADA,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1959-09-11,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,2,Temenggung Oyong Lawai Jau,NaN,SNAP,Dipilih oleh Dewan Undangan Negeri,Sarawak,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1968-10-22,NaT
104,2,Tuanku Bujang Tuanku Othman,NaN,BARJASA,Dipilih oleh Dewan Undangan Negeri,Sarawak,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1963-11-02,NaT
105,2,Tuanku Bujang Tuanku Othman,NaN,BARJASA,Dipilih oleh Dewan Undangan Negeri,Sarawak,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1965-12-18,NaT
106,2,Ahmad Taff,NaN,Perikatan - UMNO,Dipilih oleh Dewan Undangan Negeri,Singapura,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1963-11-02,1965-08-08


## Ke 3 - 11


In [208]:
import pandas as pd
from io import StringIO
from datetime import datetime, timezone, timedelta
import html

lantikan_ydpa = {
    "vol":[],
    "nama": [],
    "parti": [],
    "penggal": [],
    "penggal_mula": [],
    "penggal_tamat": [],
    "catatan": []
}


for i,link in enumerate(links[2:12],start=3):
    parliament_cycle = i
    if parliament_cycle == 9:
        continue
    
    print(link)

    with HTMLSession() as sess:
        r = sess.get(link)
        lantikans = [x for x in r.html.xpath('.//table//caption//text()') if x.strip() != ""]
        
        table_2 = r.html.xpath('.//table')[1]
        
        lantikan_ydpa = pd.read_html(StringIO(table_2.html))[0]
        
        table_2 = r.html.xpath('.//table')[0]
        
        lantikan_dun = pd.read_html(StringIO(table_2.html))[0]




    lantikan_ydpa = lantikan_ydpa.drop(axis=0, columns=["Unnamed: 2"])
    # lantikan_ydpa = lantikan_ydpa.loc[lantikan_ydpa.index != 41]
    lantikan_ydpa['lantikan'] = 'Dilantik Yang di-Pertuan Agong'
    lantikan_ydpa.columns = ['nama','parti','tempoh_mula','tempoh_tamat','lantikan']
    lantikan_ydpa['catatan'] = None
    lantikan_ydpa['negeri'] = None
    lantikan_ydpa['jawatan_parlimen'] = 'Ahli Dewan Negara'
    lantikan_ydpa['jawatan_kabinet'] = None
    lantikan_ydpa.reindex(columns=['negeri','name','parti','penggal','tempoh_mula','tempoh_tamat','lantikan','catatan'])
    lantikan_ydpa


    # lantikan_dun = lantikan_dun.drop(axis=0, columns=["No."])
    # lantikan_dun = lantikan_dun.iloc[lantikan_dun.index != 27]
    lantikan_dun['lantikan'] = 'Dipilih oleh Dewan Undangan Negeri'
    lantikan_dun.columns = ['nama','parti','negeri','tempoh_mula','tempoh_tamat','lantikan']
    lantikan_dun['catatan'] = None
    lantikan_dun['jawatan_parlimen'] = 'Ahli Dewan Negara'
    lantikan_dun['jawatan_kabinet'] = None
    lantikan_dun.reindex(columns=['negeri','name','parti','penggal','tempoh_mula','tempoh_tamat','lantikan','catatan'])

    lantikan_dun

    combined_data = pd.concat([lantikan_ydpa, lantikan_dun], axis=0)
    # combined_data = combined_data.reset_index(drop=True).drop(axis=0, columns=["catatan","penggal"])
    combined_data['vol'] = parliament_cycle
    combined_data[['parti', 'negeri']] = combined_data[['parti', 'negeri']].fillna('TIADA')


    combined_data['parti'] = combined_data['parti'].apply(
        lambda x: format_parti(x)
    )

    combined_data['nama'] = combined_data['nama'].apply(
        lambda x: re.sub(r'\[\d+\]', '', x).strip()
    )

    combined_data['tempoh_mula'] = combined_data['tempoh_mula'].apply(date_parser)
    combined_data['tempoh_tamat'] = combined_data['tempoh_tamat'].apply(date_parser)

    combined_data_top = combined_data[combined_data.groupby('nama')['nama'].transform('count') > 1].dropna(subset=['tempoh_mula', 'tempoh_tamat'], how='all')
    combined_data_bottom = combined_data[combined_data.groupby('nama')['nama'].transform('count') == 1]

    combined_data = pd.concat([combined_data_top, combined_data_bottom], axis=0).reset_index(drop=True)

    combined_data = combined_data.reindex(
        columns=[
            'vol','nama','gelaran','parti','lantikan',
            'negeri','kawasan','kod_kawasan',
            'tahun_lahir','jantina','kumpulan_etnik',
            'jawatan_kabinet','jawatan_parlimen',
            'penggal','tempoh','tempoh_mula','tempoh_tamat'
        ]
    )
    combined_data.to_csv(f"./data/dn_wiki/dn_{parliament_cycle}.csv", index=False)
    combined_data


https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-3
https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-4
https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-5
https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-6
https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-7
https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-8
https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-10
https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-11


## 9

In [209]:
import pandas as pd
from io import StringIO
from datetime import datetime, timezone, timedelta
import html

lantikan_ydpa = {
    "vol":[],
    "nama": [],
    "parti": [],
    "penggal": [],
    "penggal_mula": [],
    "penggal_tamat": [],
    "catatan": []
}


parliament_cycle = 9
link = links[parliament_cycle-1]
print(link)

with HTMLSession() as sess:
    r = sess.get(link)
    lantikans = [x for x in r.html.xpath('.//table//caption//text()') if x.strip() != ""]
    
    table_2 = r.html.xpath('.//table')[1]
    
    lantikan_ydpa = pd.read_html(StringIO(table_2.html))[0]
    
    table_2 = r.html.xpath('.//table')[0]
    
    lantikan_dun = pd.read_html(StringIO(table_2.html))[0]




lantikan_ydpa = lantikan_ydpa.drop(axis=0, columns=["No."])
# lantikan_ydpa = lantikan_ydpa.loc[lantikan_ydpa.index != 41]
lantikan_ydpa['lantikan'] = 'Dilantik Yang di-Pertuan Agong'
lantikan_ydpa.columns = ['nama','parti','tempoh_mula','tempoh_tamat','catatan','lantikan']
lantikan_ydpa['jawatan_parlimen'] ='Ahli Dewan Negara'
lantikan_ydpa['jawatan_kabinet'] = lantikan_ydpa['catatan']
lantikan_ydpa['catatan'] = None
lantikan_ydpa['negeri'] = None
lantikan_ydpa.reindex(columns=['negeri','name','parti','penggal','tempoh_mula','tempoh_tamat','lantikan','catatan','jawatan_parlimen','jawatan_kabinet'])
# lantikan_ydpa


lantikan_dun = lantikan_dun.drop(axis=0, columns=["No."])
# lantikan_dun = lantikan_dun.iloc[lantikan_dun.index != 27]
lantikan_dun['lantikan'] = 'Dipilih oleh Dewan Undangan Negeri'
lantikan_dun.columns = ['negeri','nama','parti','penggal','tempoh_mula','tempoh_tamat','lantikan']
lantikan_dun['catatan'] = None
lantikan_dun.reindex(columns=['negeri','name','parti','penggal','tempoh_mula','tempoh_tamat','lantikan','catatan'])
lantikan_dun['jawatan_parlimen'] = 'Ahli Dewan Negara'
lantikan_dun['jawatan_kabinet'] = None
lantikan_dun

combined_data = pd.concat([lantikan_ydpa, lantikan_dun], axis=0)
# combined_data = combined_data.reset_index(drop=True).drop(axis=0, columns=["catatan","penggal"])
combined_data['vol'] = parliament_cycle
combined_data[['parti', 'negeri']] = combined_data[['parti', 'negeri']].fillna('TIADA')


combined_data['parti'] = combined_data['parti'].apply(
    lambda x: format_parti(x)
)

combined_data = combined_data.dropna(subset=['nama'])
combined_data['nama'] = combined_data['nama'].dropna().apply(
    lambda x: re.sub(r'\[\d+\]', '', x).strip()
)

combined_data['tempoh_mula'] = combined_data['tempoh_mula'].apply(date_parser)
combined_data['tempoh_tamat'] = combined_data['tempoh_tamat'].apply(date_parser)

combined_data_top = combined_data[combined_data.groupby('nama')['nama'].transform('count') > 1].dropna(subset=['tempoh_mula', 'tempoh_tamat'], how='all')
combined_data_bottom = combined_data[combined_data.groupby('nama')['nama'].transform('count') == 1]

combined_data = pd.concat([combined_data_top, combined_data_bottom], axis=0).reset_index(drop=True)

combined_data = combined_data.reindex(
    columns=[
        'vol','nama','gelaran','parti','lantikan',
        'negeri','kawasan','kod_kawasan',
        'tahun_lahir','jantina','kumpulan_etnik',
        'jawatan_kabinet','jawatan_parlimen',
        'penggal','tempoh','tempoh_mula','tempoh_tamat'
    ]
)

combined_data.to_csv(f"./data/dn_wiki/dn_{parliament_cycle}.csv", index=False)
combined_data

https://ms.wikipedia.org/wiki/Ahli_Dewan_Negara,_Parlimen_Malaysia_ke-9


,vol,nama,gelaran,parti,lantikan,negeri,kawasan,kod_kawasan,tahun_lahir,jantina,kumpulan_etnik,jawatan_kabinet,jawatan_parlimen,penggal,tempoh,tempoh_mula,tempoh_tamat
0,9,Mohamed Yaacob,NaN,BN - UMNO,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,Yang di-Pertua Dewan,Ahli Dewan Negara,NaN,NaN,1996-12-10,1999-12-09
1,9,Michael Chen Wing Sum,NaN,BN - Gerakan,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,Timbalan Yang di-Pertua Dewan,Ahli Dewan Negara,NaN,NaN,1997-04-07,2000-04-06
2,9,Mohd Khalil Yaakob,NaN,BN - UMNO,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,Menteri Kabinet,Ahli Dewan Negara,NaN,NaN,1999-05-24,2002-05-23
3,9,Kasitah Gaddam,NaN,BN - UMNO,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,Menteri Kabinet,Ahli Dewan Negara,NaN,NaN,NaT,NaT
4,9,Ibrahim Ali,NaN,BN - UMNO,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,Timbalan Menteri Kabinet,Ahli Dewan Negara,NaN,NaN,1996-10-29,1999-10-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,9,Abdillah Abdul Rahim,NaN,BN - PBB,Dipilih oleh Dewan Undangan Negeri,Sarawak,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1996-12-11,1999-12-10
62,9,Janggu Banyang,NaN,BN - PBB,Dipilih oleh Dewan Undangan Negeri,Sarawak,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,Kedua,NaN,1996-12-11,1999-12-10
63,9,Ghazi @ Hasbullah Ramli,NaN,BN - UMNO,Dipilih oleh Dewan Undangan Negeri,Selangor,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1997-08-11,2000-08-10
64,9,Tee Thiong Hock,NaN,BN - MCA,Dipilih oleh Dewan Undangan Negeri,Selangor,NaN,NaN,NaN,NaN,NaN,None,Ahli Dewan Negara,NaN,NaN,1997-08-11,2000-08-10


In [211]:
import os
import pandas as pd
dn_arkib_df = pd.DataFrame()

for file in os.listdir("./data/dn_wiki/"):
    if file.endswith(".csv"):
        print(file)
        temp_df = pd.read_csv(f"./data/dn_wiki/{file}")
    
    dn_arkib_df = pd.concat([dn_arkib_df, temp_df], axis=0)


dn_arkib_df = dn_arkib_df.sort_values(by=['nama','vol']).reset_index(drop=True)
dn_arkib_df['nama'] = dn_arkib_df['nama'].apply(lambda x: re.sub(r'\(\w+\s*\w+\)', '', x).strip())
dn_arkib_df['gelaran'] = None

dn_arkib_df

dn_10.csv
dn_11.csv
dn_5.csv
dn_4.csv
dn_6.csv
dn_7.csv
dn_3.csv
dn_2.csv
dn_1.csv
dn_9.csv
dn_8.csv


,vol,nama,gelaran,parti,lantikan,negeri,kawasan,kod_kawasan,tahun_lahir,jantina,kumpulan_etnik,jawatan_kabinet,jawatan_parlimen,penggal,tempoh,tempoh_mula,tempoh_tamat
0,2,A. Arunasalam,None,Perikatan - MIC,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,NaN,Ahli Dewan Negara,NaN,NaN,1964-09-11,NaN
1,3,A. Arunasalam,None,Perikatan - MIC,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,NaN,Ahli Dewan Negara,NaN,NaN,1964-09-11,1970-09-10
2,3,A. Arunasalam,None,Perikatan - MIC,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,NaN,Ahli Dewan Negara,NaN,NaN,1971-02-20,1977-02-19
3,4,A. Arunasalam,None,BN - MIC,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,NaN,Ahli Dewan Negara,NaN,NaN,1964-09-11,1967-09-10
4,4,A. Arunasalam,None,BN - MIC,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,NaN,Ahli Dewan Negara,NaN,NaN,1971-02-20,1974-02-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1285,7,Zakaria Yahya,None,BN - UMNO,Dipilih oleh Dewan Undangan Negeri,Selangor,NaN,NaN,NaN,NaN,NaN,NaN,Ahli Dewan Negara,NaN,NaN,1984-07-30,1987-07-29
1286,7,Zakaria Yahya,None,BN - UMNO,Dipilih oleh Dewan Undangan Negeri,Selangor,NaN,NaN,NaN,NaN,NaN,NaN,Ahli Dewan Negara,NaN,NaN,1987-12-08,1990-12-07
1287,8,Zaleha Husin,None,BN - UMNO,Dilantik Yang di-Pertuan Agong,TIADA,NaN,NaN,NaN,NaN,NaN,NaN,Ahli Dewan Negara,NaN,NaN,1992-04-13,1995-04-12
1288,4,Zawiah Abdullah,None,BN - UMNO,Dipilih oleh Dewan Undangan Negeri,Terengganu,NaN,NaN,NaN,NaN,NaN,NaN,Ahli Dewan Negara,NaN,NaN,1975-01-06,1978-01-05


In [217]:
dn_12_15 = pd.read_csv("./data/ahli_dewan_mod.csv")
dn_12_15['tempoh_mula'] = pd.to_datetime(dn_12_15['tempoh_mula'], format="%d.%m.%Y", errors='coerce').dt.strftime("%Y-%m-%d")
dn_12_15['tempoh_tamat'] = pd.to_datetime(dn_12_15['tempoh_tamat'], format="%d.%m.%Y", errors='coerce').dt.strftime("%Y-%m-%d")


dn_completed = pd.concat([dn_12_15, dn_arkib_df], axis=0).reset_index(drop=True)

dn_completed = dn_completed.reindex(
    columns=[
        'vol','nama','gelaran','parti','lantikan',
        'negeri','kawasan','kod_kawasan',
        'tahun_lahir','jantina','kumpulan_etnik',
        'jawatan_kabinet','jawatan_parlimen',
        'penggal','tempoh','tempoh_mula','tempoh_tamat'
    ]
)

dn_completed = dn_completed.sort_values(by=['nama','vol']).reset_index(drop=True)
dn_completed[['gelaran','parti','negeri','jawatan_kabinet']] = dn_completed[['gelaran','parti','negeri','jawatan_kabinet']].fillna("TIADA")
dn_completed.to_csv("./data/dn_completed.csv", index=False)

In [237]:
from fuzzywuzzy import fuzz, process

MNAUAL_OVERIDES = {
    "J. E. S. Crawford" : "James Edward Seymour Crawford",
}

def clean_name(name):
    """Clean the name by removing unwanted characters"""
    return re.sub(r'\[\d+\]', '', name).strip()

def group_similar_names(names, threshold=95):
    """Group similar names using fuzzy matching"""
    cleaned_names = [clean_name(name) for name in names if pd.notna(name)]
    grouped_names = {}
    processed = set()
    
    for name in cleaned_names:
        if name in processed:
            continue
            
        matches = process.extract(name, cleaned_names, scorer=fuzz.token_set_ratio)
        similar_names = [match[0] for match in matches if match[1] >= threshold]
        
        # Use the most common original form as the canonical name
        canonical = names[cleaned_names.index(name)]
        for similar in similar_names:
            grouped_names[similar] = canonical
            processed.add(similar)
    
    return grouped_names

name_groups = group_similar_names(list(unique_names.values))
pprint(name_groups)

# Create mapping for original names to canonical forms
canonical_mapping = {}
for orig_name in dn_completed['nama'].dropna().unique():
    cleaned = clean_name(orig_name)
    if cleaned in name_groups:
        canonical_mapping[orig_name] = name_groups[cleaned]
    else:
        canonical_mapping[orig_name] = orig_name

# Create ID mapping using canonical names
canonical_names = set(canonical_mapping.values())
id_mapping = {name: i+1 for i, name in enumerate(sorted(canonical_names))}

print(canonical_mapping)
print(canonical_names)
print(id_mapping)

{'A. Arunasalam': 'A. Arunasalam',
 'A. M. Abu Bakar': 'A. M. Abu Bakar',
 'A. Shaik Dawood Abu Bakar': 'A. Shaik Dawood Abu Bakar',
 'Abang Ahmad Urai': 'Abang Ahmad Urai',
 'Abang Mustapha Abang Moasili': 'Abang Mustapha Abang Moasili',
 'Abbas Mohamed': 'Abbas Mohamed',
 'Abdillah Abdul Rahim': 'Abdillah Abdul Rahim',
 'Abdul Aziz Abdul Rahman': 'Abdul Aziz Abdul Rahman',
 'Abdul Aziz Shamsuddin': 'Abdul Aziz Shamsuddin',
 'Abdul Aziz Tapa': 'Abdul Aziz Tapa',
 'Abdul Ghafar Baba': 'Abdul Ghafar Baba',
 'Abdul Ghani Bin Mohamed Yassin': 'Abdul Ghani Bin Mohamed Yassin',
 'Abdul Ghani Othman': 'Abdul Ghani Othman',
 'Abdul Halim bin Abd. Samad': 'Abdul Halim bin Abd. Samad',
 'Abdul Halim bin Haji Suleiman': 'Abdul Halim bin Haji Suleiman',
 'Abdul Hamid Bidin': 'Abdul Hamid Bidin',
 'Abdul Hamid Ibrahim': 'Abdul Hamid Ibrahim',
 'Abdul Hamid Khan': 'Abdul Hamid Khan',
 'Abdul Hamid Latif': 'Abdul Hamid Latif',
 'Abdul Hamid Mahmud': 'Abdul Hamid Mahmud',
 'Abdul Hamid Othman': 'Abdu

In [247]:
from fuzzywuzzy import fuzz, process
import pandas as pd
import re
from pprint import pprint

# Manual overrides for known variations
MANUAL_OVERRIDES = {
    "J. E. S. Crawford": "James Edward Seymour Crawford",
    "Athi Nahappan": "Athi Nahappan",
    "Janaky Athi Nahappan": "Janaky Athi Nahappan",
    "Lingeshwaran a/l Dato' Seri R. Arunasalam": "Lingeshwaran a/l Dato' Seri R. Arunasalam"
    # Add more as needed
}

def clean_name(name):
    """Clean name and apply manual overrides"""
    if pd.isna(name):
        return name
    cleaned = re.sub(r'\[\d+\]', '', str(name)).strip()
    return MANUAL_OVERRIDES.get(cleaned, cleaned)

def group_similar_names(names, threshold=95):
    """Group similar names using fuzzy matching"""
    cleaned_names = [clean_name(name) for name in names if pd.notna(name)]
    grouped_names = {}
    processed = set()
    
    # First, process all manual overrides
    for i, name in enumerate(names):
        if pd.notna(name) and name in MANUAL_OVERRIDES:
            grouped_names[name] = MANUAL_OVERRIDES[name]
            processed.add(clean_name(name))
    
    # Then do fuzzy matching for remaining names
    for i, name in enumerate(names):
        cleaned = clean_name(name)
        if pd.isna(name) or cleaned in processed:
            continue
            
        matches = process.extract(cleaned, cleaned_names, scorer=fuzz.token_set_ratio)
        similar_names = [names[cleaned_names.index(m[0])] for m in matches if m[1] >= threshold]
        
        for similar in similar_names:
            if clean_name(similar) not in processed:
                grouped_names[similar] = name
                processed.add(clean_name(similar))
    
    return grouped_names

# Get unique names
unique_names = dn_completed['nama'].drop_duplicates().dropna()

# Group similar names
name_groups = group_similar_names(list(unique_names.values))
print("Name Groups:")
pprint(name_groups)

# Create mappings
canonical_mapping = {}
for orig_name in dn_completed['nama'].dropna().unique():
    cleaned = clean_name(orig_name)
    if cleaned in name_groups:
        canonical_mapping[orig_name] = name_groups[cleaned]
    else:
        canonical_mapping[orig_name] = orig_name

canonical_names = set(canonical_mapping.values())
id_mapping = {name: i+1 for i, name in enumerate(sorted(canonical_names))}

print("\nID Mapping:")
pprint(id_mapping)

# Apply ID mapping to DataFrame
dn_completed['author_id'] = dn_completed['nama'].map(lambda x: id_mapping.get(canonical_mapping.get(x, x)))

Name Groups:
{'A. Arunasalam': 'A. Arunasalam',
 'A. M. Abu Bakar': 'A. M. Abu Bakar',
 'A. Shaik Dawood Abu Bakar': 'A. Shaik Dawood Abu Bakar',
 'Abang Ahmad Urai': 'Abang Ahmad Urai',
 'Abang Mustapha Abang Moasili': 'Abang Mustapha Abang Moasili',
 'Abbas Mohamed': 'Abbas Mohamed',
 'Abdillah Abdul Rahim': 'Abdillah Abdul Rahim',
 'Abdul Aziz Abdul Rahman': 'Abdul Aziz Abdul Rahman',
 'Abdul Aziz Shamsuddin': 'Abdul Aziz Shamsuddin',
 'Abdul Aziz Tapa': 'Abdul Aziz Tapa',
 'Abdul Ghafar Baba': 'Abdul Ghafar Baba',
 'Abdul Ghani Bin Mohamed Yassin': 'Abdul Ghani Bin Mohamed Yassin',
 'Abdul Ghani Othman': 'Abdul Ghani Othman',
 'Abdul Halim bin Abd. Samad': 'Abdul Halim bin Abd. Samad',
 'Abdul Halim bin Haji Suleiman': 'Abdul Halim bin Haji Suleiman',
 'Abdul Hamid Bidin': 'Abdul Hamid Bidin',
 'Abdul Hamid Ibrahim': 'Abdul Hamid Ibrahim',
 'Abdul Hamid Khan': 'Abdul Hamid Khan',
 'Abdul Hamid Latif': 'Abdul Hamid Latif',
 'Abdul Hamid Mahmud': 'Abdul Hamid Mahmud',
 'Abdul Hamid O

In [249]:
# Reorder columns as specified
dn_completed = dn_completed.reindex(
    columns=[
        'author_id',
        'vol','nama','gelaran','parti','lantikan',
        'negeri','kawasan','kod_kawasan',
        'tahun_lahir','jantina','kumpulan_etnik',
        'jawatan_kabinet','jawatan_parlimen',
        'penggal','tempoh','tempoh_mula','tempoh_tamat'
    ]
)


dn_completed.to_csv("./data/dn_cleaned_w_author_id.csv", index=False)